# Import libraries

In [1]:
#---------------------------------------------------------------------------------------------------------------
#IMPORT LIBRARIES


import os
import inro.emme.desktop.app as _app #import inro.emme.desktop as _app
import inro.modeller as m


# Initialize Modeller & emmebank objects

In [2]:
#Initialize modeller and emme.database
mm = m.Modeller()
from inro.emme.matrix import MatrixData
eb = mm.emmebank


# Load trip matrices from output folder

In [7]:
#launch matrix butler to export selected trip matrices to the emmebank
#matrix_list provides the list of matrices to be exported
#[[number,"mf id", 'name of matrix']]




ext_mat_list=[[1,'mf01',"am_external_hr.mdf",'2011_am_external_hr','mf06',"mf06","mf01*(1+0.02)^21",'2041_am_external_hr'],
                [2,'mf02',"dn_external_hr.mdf",'2011_dn_external_hr','mf07',"mf07","mf02*(1+0.02)^21",'2041_dn_external_hr'],
                [3,'mf03',"ev_external_hr.mdf",'2011_ev_external_hr','mf08',"mf08","mf03*(1+0.02)^21",'2041_ev_external_hr'],
                [4,'mf04',"md_external_hr.mdf",'2011_md_external_hr','mf09',"mf09","mf04*(1+0.02)^21",'2041_md_external_hr'],
                [5,'mf05',"pm_external_hr.mdf",'2011_pm_external_hr','mf10',"mf10","mf05*(1+0.02)^21",'2041_pm_external_hr']]
                

# UPDATE THE BENCHMARK PATH         
input_path = "F:\\GGHM v4 Databanks\\Prototype GGHM4\\Inputs\\Base - 2011\\Externals\\"
save_path = "F:\\GGHM v4 Databanks\\Prototype GGHM4\\Inputs\\Base - 2041 test\\Externals\\"
scen_in = 101
scen_out = 4100


#load auto & transit matrices and update matrices names
for mat in ext_mat_list:
    
    matrix_filename = os.path.join(input_path,mat[2]) #save matrix file name into a variable
    md = MatrixData.load(matrix_filename)                 #load matrix from file
    
    if eb.matrix(mat[1]) is None: #Check if the matrix exists; if not, create it
        NAMESPACE = "inro.emme.data.matrix.create_matrix"
        create_matrix = mm.tool(NAMESPACE)
        new_mat = create_matrix(matrix_id=mat[1],
                        matrix_name=mat[3],
                        matrix_description=mat[3],
                        default_value=0)
        matrix = eb.matrix(mat[1])
        matrix.name = mat[3] #update matrix name
        matrix.description = mat[3] #update matrix description
        matrix.set_data(md,scenario_id=scen_in) #set matrix values into mf'x'
    else:
        matrix = eb.matrix(mat[1])
        matrix.name = mat[3] #update matrix name
        matrix.description = mat[3] #update matrix description
        matrix.set_data(md,scenario_id = scen_in) #set matrix values into mf'x'
        
#load auto & transit matrices and update matrices names
for mat in ext_mat_list:
    
    if eb.matrix(mat[4]) is None: #Check if the matrix exists; if not, create it
        NAMESPACE = "inro.emme.data.matrix.create_matrix"
        create_matrix = mm.tool(NAMESPACE)
        new_mat = create_matrix(matrix_id=mat[4],
                        matrix_name=mat[6],
                        matrix_description=mat[6],
                        default_value=0)
        NAMESPACE2 = "inro.emme.matrix_calculation.matrix_calculator"
        compute_matrix = mm.tool(NAMESPACE2)
        specification = {
    "type": "MATRIX_CALCULATION",
    "result": mat[5],
    "expression": mat[6],
    "constraint": {
		"by_zone": None,
		"by_value": None,
	}
}
        report = compute_matrix(specification)

    else:
        matrix = eb.matrix(mat[4])
        matrix.name = mat[7] #update matrix name
        matrix.description = mat[7] #update matrix description
        
        NAMESPACE2 = "inro.emme.matrix_calculation.matrix_calculator"
        compute_matrix = mm.tool(NAMESPACE2)
        specification = {
    "type": "MATRIX_CALCULATION",
    "result": mat[5],
    "expression": mat[6],
    "constraint": {
		"by_zone": None,
		"by_value": None,
	}
}
        report = compute_matrix(specification)
    
    
    #export matrix to mdf file
    Export_Filename = os.path.join(save_path,mat[7]+".mdf")
    matrix_data = eb.matrix(mat[5]).get_data(scen_out)
    matrix_data.save(Export_Filename)
        

# Export trip matrices to *.csv files for SPD partition

In [14]:
#EXPORT DEMAND MATRICES INTO EXCEL FILES 
    # demand is aggregated to the 'gg' ensemble 
    # square table formats
    
#---------------------------------------------------------------------------------------------------------------
# Export auto demand matrices in aggregated to a 'gg' ensemble - AM, midday, PM, evening & overnight
NAMESPACE = "inro.emme.data.matrix.export_matrices"
export_matrices = m.Modeller().tool(NAMESPACE)

base_scenario = mm.emmebank.scenario(scen_in)
matrices_file = os.path.join(save_path,"2011_external_auto_matrices.csv")
export_matrices(matrices=["mf01","mf02","mf03","mf04","mf05"],
                export_file=matrices_file,
                partition_aggregation = {'operator': 'sum', 'origins': u'gs', 'destinations': u'gs'},
                field_separator=',',
                export_format="PROMPT_DATA_FORMAT",
                skip_default_values=True,
                full_matrix_line_format="SQUARE",
                scenario = base_scenario)

base_scenario = mm.emmebank.scenario(scen_out)
matrices_file = os.path.join(save_path,"2041_external_auto_matrices.csv")
export_matrices(matrices=["mf06","mf07","mf08","mf09","mf10"],
                export_file=matrices_file,
                partition_aggregation = {'operator': 'sum', 'origins': u'gs', 'destinations': u'gs'},
                field_separator=',',
                export_format="PROMPT_DATA_FORMAT",
                skip_default_values=True,
                full_matrix_line_format="SQUARE",
                scenario = base_scenario)

print "GGHM External matrices exported successfully"

GGHM External matrices exported successfully
